In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

# 데이터 로드
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

# 결측치 처리
train_data['강수량(mm)'].fillna(0, inplace=True)
train_data['풍속(m/s)'].fillna(train_data['풍속(m/s)'].mean(), inplace=True)
train_data['습도(%)'].fillna(train_data['습도(%)'].mean(), inplace=True)
train_data['일조(hr)'].fillna(0, inplace=True)
train_data['일사(MJ/m2)'].fillna(0, inplace=True)
test_data['강수량(mm)'].fillna(0, inplace=True)
test_data['일조(hr)'] = 0
test_data['일사(MJ/m2)'] = 0


# 날짜 및 시간 특성 파생
train_data['일시'] = pd.to_datetime(train_data['일시'], format='%Y%m%d %H')
test_data['일시'] = pd.to_datetime(test_data['일시'], format='%Y%m%d %H')
train_data['연'] = train_data['일시'].dt.year
train_data['월'] = train_data['일시'].dt.month
train_data['일'] = train_data['일시'].dt.day
train_data['시간'] = train_data['일시'].dt.hour
test_data['연'] = test_data['일시'].dt.year
test_data['월'] = test_data['일시'].dt.month
test_data['일'] = test_data['일시'].dt.day
test_data['시간'] = test_data['일시'].dt.hour

# 필요하지 않은 컬럼 제거
train_data.drop(columns=['num_date_time', '일시'], inplace=True)
test_data.drop(columns=['num_date_time', '일시'], inplace=True)

# 특성과 라벨 분리
X_train = train_data.drop(columns=['전력소비량(kWh)'])
y_train = train_data['전력소비량(kWh)']

In [24]:
# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(test_data)

# 데이터 분할 (훈련 및 검증 세트)
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'n_estimators': [5000],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9],
}

# XGBoost 모델 생성
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)

# 그리드 서치 객체 생성
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
                           cv=3, n_jobs= -1, verbose=2, scoring='neg_root_mean_squared_error')

In [25]:
# 그리드 서치 수행
grid_search.fit(X_train_split, y_train_split, eval_set=[(X_val_split, y_val_split)], early_stopping_rounds=10, verbose=True)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", grid_search.best_params_)

# 검증 세트에서 최적 모델의 성능 평가
y_val_pred_best = grid_search.best_estimator_.predict(X_val_split)
val_rmse_best = np.sqrt(mean_squared_error(y_val_split, y_val_pred_best))
print("Validation RMSE with Best Model:", val_rmse_best)

# test 데이터에 대한 예측
y_test_pred_best = grid_search.best_estimator_.predict(X_test_scaled)
test_predictions_best = pd.DataFrame({'전력소비량(kWh)': y_test_pred_best})
test_predictions_best
# test_predictions_best.to_csv('test_predictions_best.csv', index=False)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-lea

[0]	validation_0-rmse:3423.75117
[0]	validation_0-rmse:3420.88102
[0]	validation_0-rmse:3414.65571
[0]	validation_0-rmse:3420.85917
[0]	validation_0-rmse:3420.97229
[1]	validation_0-rmse:3399.45511
[0]	validation_0-rmse:3420.74623
[0]	validation_0-rmse:3420.80108
[1]	validation_0-rmse:3396.80241
[1]	validation_0-rmse:3396.68043
[0]	validation_0-rmse:3414.57406
[1]	validation_0-rmse:3396.81772
[1]	validation_0-rmse:3384.34589
[0]	validation_0-rmse:3414.62680
[1]	validation_0-rmse:3396.33928
[1]	validation_0-rmse:3396.47040
[2]	validation_0-rmse:3381.48085
[0]	validation_0-rmse:3323.64327
[0]	validation_0-rmse:3414.58118
[0]	validation_0-rmse:3416.21640


/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-lea

[0]	validation_0-rmse:3414.67141
[0]	validation_0-rmse:3413.13289
[2]	validation_0-rmse:3378.81124
[2]	validation_0-rmse:3378.77672
[0]	validation_0-rmse:3413.14348
[0]	validation_0-rmse:3322.97749
[2]	validation_0-rmse:3378.65856
[0]	validation_0-rmse:3413.14842
[0]	validation_0-rmse:3338.09889
[0]	validation_0-rmse:3413.16192
[2]	validation_0-rmse:3378.40744
[3]	validation_0-rmse:3357.78274
[2]	validation_0-rmse:3378.24497
[1]	validation_0-rmse:3384.14769
[1]	validation_0-rmse:3210.28924
[0]	validation_0-rmse:3323.53523
[1]	validation_0-rmse:3384.32317
[0]	validation_0-rmse:3413.04476
[0]	validation_0-rmse:3300.04941
[2]	validation_0-rmse:3366.15823
[3]	validation_0-rmse:3355.23683
[1]	validation_0-rmse:3385.87914
[3]	validation_0-rmse:3355.15024
[1]	validation_0-rmse:3208.05822
[1]	validation_0-rmse:3384.26838
[1]	validation_0-rmse:3222.58073
[3]	validation_0-rmse:3355.02114
[3]	validation_0-rmse:3354.67573
[0]	validation_0-rmse:3323.24823
[0]	validation_0-rmse:3413.13220
[2]	valida

/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/opt/miniconda3/envs/jw/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-lea

[0]	validation_0-rmse:3291.73698
[4]	validation_0-rmse:3332.07821
[4]	validation_0-rmse:3334.49398
[2]	validation_0-rmse:3127.30893
[4]	validation_0-rmse:3332.01854
[1]	validation_0-rmse:3381.34904
[0]	validation_0-rmse:3292.11011
[4]	validation_0-rmse:3331.76870
[1]	validation_0-rmse:3381.42579
[1]	validation_0-rmse:3154.29319
[3]	validation_0-rmse:3338.11762
[4]	validation_0-rmse:3331.40710
[2]	validation_0-rmse:3366.04831
[3]	validation_0-rmse:3027.58444
[2]	validation_0-rmse:3142.72427
[2]	validation_0-rmse:3142.77046
[2]	validation_0-rmse:3367.71215
[2]	validation_0-rmse:3365.85792
[1]	validation_0-rmse:3221.90292
[4]	validation_0-rmse:3331.19609
[0]	validation_0-rmse:3283.92650
[1]	validation_0-rmse:3154.61674
[0]	validation_0-rmse:3203.88028
[3]	validation_0-rmse:3023.83152
[0]	validation_0-rmse:3284.37911
[1]	validation_0-rmse:3381.48720
[1]	validation_0-rmse:3223.30671
[5]	validation_0-rmse:3314.71543
[5]	validation_0-rmse:3314.45940
[1]	validation_0-rmse:3154.53634
[1]	valida

KeyboardInterrupt: 